In [1]:
# Librerias
# Tratamiento de datos
# ==============================================================================
import pandas as pd
import numpy as np
from pandas.core.frame import DataFrame
# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns


# Preprocesado y modelado
# ==============================================================================
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.weightstats import ttest_ind
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, median_absolute_error
from pandas.api.types import is_numeric_dtype
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
sns.set_context("talk")

In [4]:
df=pd.read_csv("C:/Users/54351/Downloads/Base 5 Incumplimiento de préstamo(2).csv")

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/54351/Downloads/Base 5 Incumplimiento de préstamo(2).csv'

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# Eliminamos columnas innecesarias
cols_to_drop = ['Unnamed: 0', 'year']
df = df.drop(columns=cols_to_drop)



In [ ]:
## Separo para posteriormente hacer el dummy

categorical=[
    "Gender","approv_in_adv","loan_type","loan_purpose","loan_purpose"
    ,"Credit_Worthiness","open_credit","business_or_commercial","Neg_ammortization"
    ,"interest_only","lump_sum_payment","construction_type","occupancy_type","Secured_by"
    ,"total_units","credit_type","co-applicant_credit_type","age","submission_of_application"
    ,"Region","Security_Type","loan_limit"
]
numeric =[
    "loan_amount","term","income","Credit_Score","Upfront_charges","LTV","property_value"
    ,"dtir1","Status"
]

In [ ]:
 # Impugnamos valores nulos en las variables categóricas, reemplazándolos
    # por la moda
for col in categorical:
    df[col] = df[col].fillna(df[col].mode().values[0])
    # Impugnamos valores nulos en las variables numéricas, reemplazándolos
    # por la mediana
for col in numeric:
    df[col] = df[col].fillna(df[col].median())


    # Verificamos que no existan valores nulos en el dataset final
assert df.isna().any().any() == False

    # Verificamos que todas las columnas numéricas son variables numéricas
assert np.all(is_numeric_dtype(df[col]) for col in df.columns)


In [ ]:
df.info()

In [ ]:
encoded_categorias=pd.get_dummies(df[categorical])
df=pd.concat([encoded_categorias,df[numeric]], axis=1)

In [ ]:
df.head()

In [ ]:
X=df.drop(["Status"],axis = 1) #variables clasificadoras (features)
y=df.Status # grupo (target)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42, stratify=y)
pd.value_counts(df['Status'])

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import fbeta_score, accuracy_score, f1_score, precision_score, recall_score  # métricas para evaluar
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
!pip install lightgbm
from lightgbm import LGBMClassifier


In [ ]:
## tarda demasasiadoooooo no volver a correr
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
# en este caso proponemos una búsqueda de la mejor estimación del modelo logístico armando una grilla dbe parámetros
# definimos en solver distintos métodos de estimación posibles de la logística que permite la función LogisticRegression  del paquete sklearn.linear
solver = ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga']
# y en C se definen parámetros de regularización que aceleran el proceso de estimación (a medida que son más grandes) en este caso se proponen un vector de valores
# aleatorios espaciados según una escala logarítmica ( podrían haberse seleccionado en forma uniforme que son los nros aleatorios que se conocen comunmente).
C = np.logspace(-2, 10, 13)
# definimos el modelo a estimar LogisticRegression previo estandarizar los datos con StandardScaler
pipe_LGBM = Pipeline([('sc', StandardScaler()),
    ('LGBM', LGBMClassifier())
    ])
# definimos los vactores de parámetros para hacer las estimaciones
params_lr= {
    'LGBM__n_estimators': [5, 7, 10, 12, 15],
    'LGBM__max_depth': [5, 7, 10, 12, 15, 20],
    'LGBM__learning_rate': [0.01, 0.1, 0.25, 0.5, 0.75, 1],
    'LGBM__random_state': [11]
    }
# proponemos la busqueda del mejor modelo según el resultado de la mejor clasificación (accuracy) con la tasa crosvalidada (GridSearchCV)
search_lr = GridSearchCV(estimator=pipe_LGBM,
                      param_grid=params_lr,
                      cv = 5,
                      scoring="roc_auc",
                      return_train_score=True)

search_lr.fit(X_train, y_train)
print(f" Best score is: {search_lr.best_score_} with parameters: {search_lr.best_params_}")

In [ ]:
##hiperparametros optimizados pero no se usan ya que la mejora no es significativa
#params = {
#    'learning_rate': 0.25,
#    'max_depth': 20,
#    'n_estimators': 15,
#    'random_state': 11,
#}

pipe_LGBM = Pipeline([('sc', StandardScaler()),
    ('LGBM', LGBMClassifier())])
pipe_LGBM.fit(X_train,y_train)
y_pred_lr = pipe_LGBM.predict(X_test)
y_pred_train = pipe_LGBM.predict(X_train)

In [ ]:
pip install tabulate


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import fbeta_score, accuracy_score, f1_score, precision_score, recall_score  # métricas para evaluar
from tabulate import tabulate

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
print('--------------------')
print('Tabla de confusión en train')
print('-------------------')
CM_train = confusion_matrix(y_train,y_pred_train)  # Matriz de confusión en train
print(CM_train)
print('--------------------')
print('Reporte de medidas de desempeño en train')
print('-------------------')
print(classification_report(y_train,y_pred_train, digits = 2))
print('accuracy: ', accuracy_score(y_train,y_pred_train))
classification_rep2 = classification_report(y_train,y_pred_train, digits=2, output_dict=True)
print('--------------------')
print('Tabla de confusión en test')
print('-------------------')
CM = confusion_matrix(y_test,y_pred_lr)  # Matriz de confusión en test
print(CM)
print('--------------------')
print('Reporte de medidas de desempeño')
print('-------------------')
print(classification_report(y_test,y_pred_lr, digits = 2))
print('accuracy: ', accuracy_score(y_test,y_pred_lr))
classification_rep1 = classification_report(y_test, y_pred_lr, digits=2, output_dict=True)

print('-------------------')
print('Resumen de medidas en train ')
print('-------------------')
# Convierte el informe en una tabla con tabulate

print('-------------------')
print('Resumen de medidas en test')
print('-------------------')
table = [["Clase", "Precisión", "Recuperación", "F1-Score", "Soporte"]]
for label, scores in classification_rep1.items():
    if label.isnumeric():
        table.append([label, scores['precision'], scores['recall'], scores['f1-score'], scores['support']])
# Muestra la tabla con colores y formato
print(tabulate(table, headers="firstrow", tablefmt="fancy_grid", numalign="center", stralign="center"))


# Imprime la exactitud
print('Exactitud:', accuracy_score(y_test, y_pred_lr))



# Convierte el informe en una tabla con tabulate
table = [["Clase", "Precisión", "Recuperación", "F1-Score", "Soporte"]]
for label, scores in classification_rep2.items():
    if label.isnumeric():
        table.append([label, scores['precision'], scores['recall'], scores['f1-score'], scores['support']])

# Muestra la tabla con colores y formato
print(tabulate(table, headers="firstrow", tablefmt="fancy_grid", numalign="center", stralign="center"))

# Imprime la exactitud
print('Exactitud:', accuracy_score(y_train, y_pred_train))

In [ ]:
from sklearn.metrics import RocCurveDisplay
log_cor_est = RocCurveDisplay.from_estimator(pipe_LGBM,X_test, y_test,)
plt.show()

In [ ]:
## no me toma el codigo

features_trasformed=list(X.columns.values)
fi=list(pipe_LGBM["LGBM"].feature_importances_)

feature_importance_df=pd.DataFrame()
feature_importance_df["feature"]=features_trasformed
feature_importance_df["feature_importance"]=fi
feature_importance_df=feature_importance_df.sort_values(by="feature_importance",ascending=False)
feature_importance_df.head(20)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Crear una figura con subplots 2x3
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Lista de las columnas a graficar
columnas = ["Credit_Score", "LTV", "income", "dtir1", "loan_amount"]

# Lista de etiquetas de las columnas
etiquetas = ["Credit Score", "LTV", "Income", "DTIR1", "Loan Amount"]

# Recorrido para crear los boxplots
for i, columna in enumerate(columnas):
    row = i // 3
    col = i % 3
    sns.boxplot(data=df, x="Status", y=columna, ax=axes[row, col], showfliers=False)
    axes[row, col].set_title(f"{etiquetas[i]} vs. Status")

# Ajustar los espacios entre los subplots
plt.tight_layout()

# Mostrar la figura con los boxplots en una matriz 2x3
plt.show()

In [ ]:
## Ahora realizamos un analisis de las variables clasificatorias
## importamos la base limpia sin realizar el dummy para hacer el analisis con variables clasificatoria
df1=pd.read_csv("C:/Users/54351/Downloads/Base 5 Incumplimiento de préstamo.csv")

cols_to_drop = ['Unnamed: 0', 'year']
df1 = df1.drop(columns=cols_to_drop)

categorical=[
    "Gender","approv_in_adv","loan_type","loan_purpose","loan_purpose"
    ,"Credit_Worthiness","open_credit","business_or_commercial","Neg_ammortization"
    ,"interest_only","lump_sum_payment","construction_type","occupancy_type","Secured_by"
    ,"total_units","credit_type","co-applicant_credit_type","age","submission_of_application"
    ,"Region","Security_Type","loan_limit"
]
numeric =[
    "loan_amount","term","income","Credit_Score","Upfront_charges","LTV","property_value"
    ,"dtir1","Status"
]
 # Impugnamos valores nulos en las variables categóricas, reemplazándolos
    # por la moda
for col in categorical:
    df1[col] = df1[col].fillna(df1[col].mode().values[0])
    # Impugnamos valores nulos en las variables numéricas, reemplazándolos
    # por la mediana
for col in numeric:
    df1[col] = df1[col].fillna(df1[col].median())


    # Verificamos que no existan valores nulos en el dataset final
assert df1.isna().any().any() == False

    # Verificamos que todas las columnas numéricas son variables numéricas
assert np.all(is_numeric_dtype(df[col]) for col in df1.columns)
df1.info()

In [ ]:


# Crear una figura con subplots 3x3
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Lista de columnas a graficar
columnas = ["credit_type", "Credit_Worthiness", "Neg_ammortization", "occupancy_type", "loan_purpose"]

# Recorrido para crear los gráficos en la misma fila
for i, columna in enumerate(columnas):
    row = i // 3
    col = i % 3
    sns.histplot(df1, x=columna, hue="Status", stat="probability", multiple="fill", ax=axes[row, col])
    axes[row, col].set_title(f'{columna} vs. Status')

# Ajustar los espacios entre los subplots
plt.tight_layout()

# Mostrar la figura con los gráficos en una matriz 3x3
plt.show()